In [1]:
import pandas as pd
import glob
import xml.etree.ElementTree as ET

In [2]:
x1 = pd.read_xml('./busstop_output.xml')

In [3]:
tree = ET.parse('busstop_output.xml')
root = tree.getroot()

In [4]:
[elem.tag for elem in root.iter()]

['stops',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 'stopinfo',
 '

In [5]:
df = pd.DataFrame()
cnt = 0
for stop in root.iter('stops'):
    for stpinf in stop.findall('stopinfo'):
        # print(stpinf.attrib)
        df.loc[cnt, 'id'] = stpinf.attrib['id']
        df.loc[cnt, 'type'] = stpinf.attrib['type']
        try:df.loc[cnt, 'busStop'] = stpinf.attrib['busStop']
        except:pass
        df.loc[cnt, 'lane'] = stpinf.attrib['lane']
        df.loc[cnt, 'pos'] = stpinf.attrib['pos']
        df.loc[cnt, 'parking'] = stpinf.attrib['parking']
        df.loc[cnt, 'started'] = stpinf.attrib['started']
        df.loc[cnt, 'ended'] = stpinf.attrib['ended']
        df.loc[cnt, 'delay'] = stpinf.attrib['delay']
        try:df.loc[cnt, 'arrivalDelay'] = stpinf.attrib['arrivalDelay']
        except:pass
        df.loc[cnt, 'initialPersons'] = stpinf.attrib['initialPersons']
        df.loc[cnt, 'loadedPersons'] = stpinf.attrib['loadedPersons']
        df.loc[cnt, 'unloadedPersons'] = stpinf.attrib['unloadedPersons']
        df.loc[cnt, 'initialContainers'] = stpinf.attrib['initialContainers']
        df.loc[cnt, 'loadedContainers'] = stpinf.attrib['loadedContainers']
        df.loc[cnt, 'unloadedContainers'] = stpinf.attrib['unloadedContainers']
        cnt+=1

In [7]:
df.busStop = df.busStop.astype(str)
df.busStop = df.busStop.apply(lambda x: x.split('-')[0])
df = df.sort_values(by=['id', 'started'])
df.started = df.started.apply(lambda x: int(float(x)))

In [8]:
def time_conv(x):
    h, m, s = x.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

In [6]:
gtfs_data = pd.read_csv('./trips_vid_time_stops_GTFS_20220111.csv')
gtfs_data = gtfs_data.dropna()
gtfs_data.first_stop_id = gtfs_data.first_stop_id.astype(int).astype(str)
gtfs_data.last_stop_id = gtfs_data.last_stop_id.astype(int).astype(str)
gtfs_data.vid = gtfs_data.vid.astype(str)

In [9]:
gtfs_data.gtfs_start_time = gtfs_data.gtfs_start_time.apply(lambda x: time_conv(x))
gtfs_data.gtfs_end_time = gtfs_data.gtfs_end_time.apply(lambda x: time_conv(x))

In [52]:
#based on start time
df2 = df[['id', 'type', 'busStop', 'started', 'delay']].merge(gtfs_data.sort_values(['vid', 'gtfs_start_time']), left_on = ['id', 'busStop'], right_on = ['vid', 'first_stop_id'], how='right')
df2['start_diff'] = abs(df2.gtfs_start_time - df2.started)
df2 = df2.sort_values(['id', 'start_diff', 'started']).reset_index(drop=True)
# df2 = df2.dropna()

In [50]:
t1 = df[df.id == '154'].sort_values('started')
t1[t1.started>67000].head(20)

,id,type,busStop,lane,pos,parking,started,ended,delay,arrivalDelay,initialPersons,loadedPersons,unloadedPersons,initialContainers,loadedContainers,unloadedContainers
20792,154,Gillig_101,277,-139744479#2_0,220.41,0,67026,67027.00,934.00,934.00,26,2,0,0,0,0
20806,154,Gillig_101,278,-139744479#0_0,123.28,0,67049,67050.00,930.00,930.00,28,0,0,0,0,0
20817,154,Gillig_101,279,-884685645_0,106.68,0,67069,67070.00,923.00,923.00,28,0,2,0,0,0
20829,154,Gillig_101,280,-51067076#1_0,136.49,0,67097,67098.00,903.00,903.00,26,1,0,0,0,0
20852,154,Gillig_101,282,45397157_0,316.16,0,67137,67138.00,851.00,851.00,27,0,0,0,0,0
20873,154,Gillig_101,283,861134578_0,61.76,0,67182,67183.00,814.00,814.00,27,0,0,0,0,0
20890,154,Gillig_101,284,-862527367#3_0,135.01,0,67228,67229.00,823.00,823.00,27,0,0,0,0,0
20898,154,Gillig_101,285,-393463231#2_0,255.90,0,67257,67258.00,770.00,770.00,27,0,1,0,0,0
20915,154,Gillig_101,286,-19500872#7_0,79.64,0,67281,67282.00,739.00,739.00,26,0,1,0,0,0
20965,154,Gillig_101,1351,862527355#0_0,129.33,0,67385,67386.00,785.00,785.00,25,0,0,0,0,0


In [62]:
df2.groupby(['trip_id']).first().reset_index()[['trip_id', 'id', 'started']].merge(df, on=['id', 'started'], how='left').to_csv('busstop_info_new.csv', index=False)

In [55]:
df2.groupby(['trip_id']).first().reset_index()

,trip_id,id,type,busStop,started,delay,vid,start_time,gtfs_start_time,gtfs_end_time,first_stop_id,last_stop_id,block_id,block_name,start_diff
0,204888020,119,Gillig_103,354,51135.0,0.00,119,20220111 08:46,31860,34200,354,95,1002,107,19275.0
1,204889020,154,Gillig_101,1351,81039.0,39.00,154,20220111 22:30,81000,82800,1351,95,2802,155,39.0
2,204890020,126,Gillig_103,1351,77402.0,5.00,126,20220111 21:32,77400,79200,1351,95,3002,156,2.0
3,204891020,154,Gillig_101,1351,73779.0,0.00,154,20220111 20:25,73800,75600,1351,95,2802,155,21.0
4,204892020,126,Gillig_103,1351,70180.0,0.00,126,20220111 19:25,70200,72000,1351,95,3002,156,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,206975020,703,Gillig_108,1566,50761.0,61.00,703,20220111 14:03,50700,51600,1566,1874,3902,3302,61.0
708,207272020,146,Gillig_107,354,48072.0,6612.00,146,20220111 11:26,41460,42900,354,756,2602,153,6612.0
709,207273020,146,Gillig_107,756,52829.0,9929.00,146,20220111 11:56,42900,44400,756,1351,2602,153,9929.0
710,207274020,147,Gillig_101,166,47014.0,216.00,147,20220111 12:55,46800,47700,166,756,102,101,214.0


In [54]:
#choosing the shortest delay between gtfs start time and sim start time as the corresponding trip
df2.groupby(['trip_id']).first().reset_index().to_csv('trips_from_xml_blocks.csv', index=False)

In [13]:
#based on end time
df3 = df[['id', 'type', 'busStop', 'started']].merge(gtfs_data.sort_values(['vid', 'gtfs_end_time']), left_on = ['id', 'busStop'], right_on = ['vid', 'first_stop_id'], how='right')
df3['end_diff'] = abs(df3.gtfs_end_time - df3.started)
df3 = df3.sort_values(['id', 'end_diff', 'started']).reset_index(drop=True)

In [139]:
df3.groupby(['trip_id']).first().reset_index().head(20)

,trip_id,id,type,busStop,started,vid,start_time,gtfs_start_time,gtfs_end_time,first_stop_id,last_stop_id,block_id,block_name,end_diff
0,204888020,119,Gillig_103,354,51135.0,119,20220111 08:46,31860,34200,354,95,1002,107,16935.0
1,204889020,154,Gillig_101,1351,81039.0,154,20220111 22:30,81000,82800,1351,95,2802,155,1761.0
2,204890020,126,Gillig_103,1351,77428.0,126,20220111 21:32,77400,79200,1351,95,3002,156,1772.0
3,204891020,154,Gillig_101,1351,73822.0,154,20220111 20:25,73800,75600,1351,95,2802,155,1778.0
4,204892020,126,Gillig_103,1351,70222.0,126,20220111 19:25,70200,72000,1351,95,3002,156,1778.0
5,204893020,154,Gillig_101,1351,67385.0,154,20220111 18:25,66600,68400,1351,95,2802,155,1015.0
6,204894020,506,Gillig_105,1351,68348.0,506,20220111 18:10,65400,67200,1351,95,5702,452,1148.0
7,204895020,126,Gillig_103,1351,63110.0,126,20220111 17:30,63000,64800,1351,95,3002,156,1690.0
8,204896020,504,Gillig_105,1351,66586.0,504,20220111 17:06,61800,63900,1351,95,1102,108,2686.0
9,204897020,146,Gillig_107,1351,60579.0,146,20220111 16:45,60600,62700,1351,95,2602,153,2121.0


In [144]:
df2[df2.vid == '119'].sort_values('started')

,id,type,busStop,started,trip_id,vid,start_time,gtfs_start_time,gtfs_end_time,first_stop_id,last_stop_id,block_id,block_name,start_diff
202,119,Gillig_103,95,42800.0,204954020,119,20220111 11:30,41700,44700,95,690,1002,107,1100.0
216,119,Gillig_103,145,43410.0,205168020,119,20220111 10:20,36300,38100,145,288,1002,107,7110.0
219,119,Gillig_103,288,45407.0,205075020,119,20220111 10:33,38100,39900,288,1351,1002,107,7307.0
220,119,Gillig_103,288,45428.0,205075020,119,20220111 10:33,38100,39900,288,1351,1002,107,7328.0
235,119,Gillig_103,1351,46475.0,206278020,119,20220111 17:04,61200,64800,1351,1876,6002,455,14725.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,119,Gillig_103,1351,82278.0,206278020,119,20220111 17:04,61200,64800,1351,1876,6002,455,21078.0
211,119,Gillig_103,1351,82278.0,206303020,119,20220111 20:55,75600,78900,1351,1876,6002,455,6678.0
247,119,Gillig_103,1351,82278.0,206289020,119,20220111 15:09,54000,57600,1351,1876,6002,455,28278.0
255,119,Gillig_103,1351,82278.0,204921020,119,20220111 11:00,39900,41700,1351,95,1002,107,42378.0
